In [1]:
import os
os.chdir('..')

In [2]:
from fox.config import Config
from fox.dataset import ComboDataset
from fox.model_yolo import Model
from fox.train import train
import matplotlib.pyplot as plt
import torch

from fox.yolov3.utils.datasets import LoadImagesAndLabels
from fox.utils import parse_data_cfg

In [3]:
c = Config(USE_PLANERCNN=False, BATCH_SIZE=2)
# d = ComboDataset(c)
# loader = torch.utils.data.DataLoader(d, batch_size=2, shuffle=True, collate_fn=d.collate_fn)
# loader_val = torch.utils.data.DataLoader(ComboDataset(c, train=False), batch_size=2, collate_fn=d.collate_fn)

def collate_fn(batch):
    yolo_data = LoadImagesAndLabels.collate_fn(batch)
    return yolo_data[0].float() / 255, None, yolo_data, None

data = parse_data_cfg("data/mini/yolo/custom.data")
d = LoadImagesAndLabels(
    data["train"],
    c.yolo_config.opt.img_size[1],
    c.BATCH_SIZE,
    augment=True,
    hyp=c.yolo_config.hyp,
    rect=c.yolo_config.opt.rect,
    cache_images=c.yolo_config.opt.cache_images,
    single_cls=False,
    mosiac=False,
    label_files_path=data["labels"]
)
loader = torch.utils.data.DataLoader(d, batch_size=c.BATCH_SIZE, shuffle=True, pin_memory=True, collate_fn=collate_fn)

loader_val = torch.utils.data.DataLoader(
    LoadImagesAndLabels(
        data["valid"],
        c.yolo_config.opt.img_size[-1],
        c.BATCH_SIZE,
        hyp=c.yolo_config.hyp,
        rect=True,
        cache_images=c.yolo_config.opt.cache_images,
        single_cls=False,
        label_files_path=data["labels"]
    ),
    batch_size=c.BATCH_SIZE, pin_memory=True, collate_fn=collate_fn
)


Caching labels data/mini/yolo/train.txt (64 found, 0 missing, 0 empty, 0 duplicate, for 64 images): 100%|██████████| 64/64 [00:00<00:00, 1408.61it/s]
Caching labels data/mini/yolo/test.txt (5 found, 0 missing, 0 empty, 0 duplicate, for 5 images): 100%|██████████| 5/5 [00:00<00:00, 1017.24it/s]


In [4]:
# model = Model(c, d.yolo_dataset.yolo_labels, len(loader)).to("cuda")
model = Model(c, d.yolo_labels, len(loader)).to("cuda")

Using cache found in /home/ouroboros/.cache/torch/hub/facebookresearch_WSL-Images_master
Loading weights:  weights/midas.pt
Model Summary: 147 layers, 5.93877e+07 parameters, 5.93877e+07 gradients
Model Summary: 147 layers, 5.93877e+07 parameters, 5.93877e+07 gradients


In [5]:
train(model, c, loader, loader_val, device="cuda")

Epoch 2:   0%|          | 0/32 [00:01<?, ?it/s, loss=0.2311, midas_loss=0.0000, yolo_loss=0.2311]


KeyboardInterrupt: 